In [1]:

# importing required libraries
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np
import nltk



In [2]:
df = pd.read_csv('../dataset/train_cleaned2.csv')


In [3]:
df.head()

,id,name,document_text,cat_name
0,22474,"['inform', 'regard', 'merger', 'navio', 'marit...","['at', 'special', 'meet', 'held', 'march', '24...",Corporate Communications
1,27460,"['announc', 'approv', 'chang', 'membership', '...","['on', 'april', '2', '2021', 'china', 'financi...",Securities Settlement
2,6926,"['sfc', 'suspend', 'shiu', 'yau', 'wah', 'five...","['the', 'secur', 'futur', 'commiss', 'sfc', 's...",Antitrust
3,6982,"['renminbi', 'rmb', 'haircut', 'februari', '4'...","['pursuant', 'section', '262', 'clear', 'hous'...",Securities Settlement
4,5022,"['antimoney', 'launder', 'counter', 'financ', ...","['money', 'launder', 'terror', 'financ', 'mltf...",Financial Crime


In [4]:
# lemmitization
from nltk.stem import WordNetLemmatizer
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in nltk.word_tokenize(text)]

In [10]:
df['name'] = df['name'].apply(lemmatize_text)

In [18]:
df['document_text'] = df['document_text'].apply(lemmatize_text)
df['document_text'] = df['document_text'].apply(lambda x: ' '.join(x))

In [16]:
df['name'] = df['name'].apply(lambda x: ' '.join(x))

In [17]:
df.head()

,id,name,document_text,cat_name
0,22474,"[ 'inform ' , 'regard ' , 'merger ' , 'navio '...","['at', 'special', 'meet', 'held', 'march', '24...",Corporate Communications
1,27460,"[ 'announc ' , 'approv ' , 'chang ' , 'members...","['on', 'april', '2', '2021', 'china', 'financi...",Securities Settlement
2,6926,"[ 'sfc ' , 'suspend ' , 'shiu ' , 'yau ' , 'wa...","['the', 'secur', 'futur', 'commiss', 'sfc', 's...",Antitrust
3,6982,"[ 'renminbi ' , 'rmb ' , 'haircut ' , 'februar...","['pursuant', 'section', '262', 'clear', 'hous'...",Securities Settlement
4,5022,"[ 'antimoney ' , 'launder ' , 'counter ' , 'fi...","['money', 'launder', 'terror', 'financ', 'mltf...",Financial Crime


In [8]:
# nltk.download('wordnet')
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Suraj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Suraj\AppData\Roaming\nltk_data...


True

In [19]:
df.head()

,id,name,document_text,cat_name
0,22474,"[ 'inform ' , 'regard ' , 'merger ' , 'navio '...","[ 'at ' , 'special ' , 'meet ' , 'held ' , 'ma...",Corporate Communications
1,27460,"[ 'announc ' , 'approv ' , 'chang ' , 'members...","[ 'on ' , 'april ' , ' 2 ' , '2021 ' , 'china ...",Securities Settlement
2,6926,"[ 'sfc ' , 'suspend ' , 'shiu ' , 'yau ' , 'wa...","[ 'the ' , 'secur ' , 'futur ' , 'commiss ' , ...",Antitrust
3,6982,"[ 'renminbi ' , 'rmb ' , 'haircut ' , 'februar...","[ 'pursuant ' , 'section ' , '262 ' , 'clear '...",Securities Settlement
4,5022,"[ 'antimoney ' , 'launder ' , 'counter ' , 'fi...","[ 'money ' , 'launder ' , 'terror ' , 'financ ...",Financial Crime


In [20]:
df.to_csv('../dataset/train_cleaned3.csv', index=False)